In [13]:
import math

import torch
import torch.nn as nn

# nn.Embedding 的基本用法

- `num_embeddings`：字典的大小。对应上面词典的大小，如果你的词典中包含5000个单词，那么这个字段就填5000
- `embedding_dim`：要将单词编码成多少维的向量
- `padding_idx`：填充索引，即，对于某个索引编码为0。字典里unknown项（未知的单词），使用该参数，将之编码成0。

https://blog.csdn.net/zhaohongfei_358/article/details/122809709

In [ ]:
embedding=nn.Embedding(20,5,padding_idx=1)
embedding(torch.LongTensor([0,1,2,3,4,5]))

## nn.Embedding 的可学习性

可以加入模型，从而学习参数。

In [ ]:
embedding = nn.Embedding(20, 5, padding_idx=3) # 对3不进行编码
optimizer = torch.optim.SGD(embedding.parameters(), lr=0.1)
criteria = nn.MSELoss()

for i in range(1000):
    outputs = embedding(torch.LongTensor([0,1,2,3,4]))
    loss = criteria(outputs, torch.ones(5, 5))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
embedding(torch.LongTensor([0,1,2,3,4]))

# 演示 `nn.Transformer` 的使用

## nn.Transformer的构造参数详解

![Alt text](pics/transformer.png)
- d_model: Encoder和Decoder输入参数的特征维度。也就是词向量的维度。默认为512
- nhead: 多头注意力机制中，head的数量。关于Attention机制，可以参考这篇文章。注意该值并不影响网络的深度和参数数量。默认值为8。
- num_encoder_layers: TransformerEncoderLayer的数量。该值越大，网络越深，网络参数量越多，计算量越大。默认值为6
- num_decoder_layers：TransformerDecoderLayer的数量。该值越大，网络越深，网络参数量越多，计算量越大。默认值为6
- dim_feedforward：Feed Forward层（Attention后面的全连接网络）的隐藏层的神经元数量。该值越大，网络参数量越多，计算量越大。默认值为2048
- dropout：dropout值。默认值为0.1
- activation： Feed Forward层的激活函数。取值可以是string(“relu” or “gelu”)或者一个一元可调用的函数。默认值是relu
- custom_encoder：自定义Encoder。若你不想用官方实现的TransformerEncoder，你可以自己实现一个。默认值为None
- custom_decoder: 自定义Decoder。若你不想用官方实现的TransformerDecoder，你可以自己实现一个。
- layer_norm_eps: Add&Norm层中，BatchNorm的eps参数值。默认为1e-5
- batch_first：batch维度是否是第一个。如果为True，则输入的shape应为(batch_size, 词数，词向量维度)，否则应为(词数, batch_size, 词向量维度)。默认为False。这个要特别注意，因为大部分人的习惯都是将batch_size放在最前面，而这个参数的默认值又是False，所以会报错。
- norm_first – 是否要先执行norm。例如，在图中的执行顺序为 Attention -> Add -> Norm。若该值为True，则执行顺序变为：Norm -> Attention -> Add。

## Transformer的forward参数详解
Transformer的forward参数需要详细解释，这里我先将其列出来，进行粗略解释，然后再逐个进行详细解释：

- src: Encoder的输入。也就是将token进行Embedding并Positional Encoding之后的tensor。必填参数。Shape为(batch_size, 词数, 词向量维度)
- tgt: 与src同理，Decoder的输入。 必填参数。Shape为(词数, 词向量维度)
- src_mask: 对src进行mask。不常用。Shape为(词数, 词数)
- tgt_mask：对tgt进行mask。常用。Shape为(词数, 词数)
- memory_mask – 对Encoder的输出memory进行mask。 不常用。Shape为(batch_size, 词数, 词数)
- src_key_padding_mask：对src的token进行mask. 常用。Shape为(batch_size, 词数)
- tgt_key_padding_mask：对tgt的token进行mask。常用。Shape为(batch_size, 词数)
- memory_key_padding_mask：对tgt的token进行mask。不常用。Shape为(batch_size, 词数)

```
上面的所有mask都是0代表不遮掩，-inf代表遮掩。另外，src_mask、tgt_mask和memory_mask是不需要传batch的。

补充：上面的说法是pytorch1.11版本。我发现1.11版本key_padding_mask可以用True/False，但1.12版本key_padding_mask好像只能是True/False，其中True表示遮掩，而False表示不遮掩（这个可不要弄混了，这个和Transformer原论文实现正好相反，如果弄反了，会造成结果为nan）。
```

## src & tgt

输入的shape() 为 (batch_size=3,sentence_len=10)，Embedding 后的 shape() 为 (1,10,embedding_dims=128)

## src_mask, tgt_mask, memory_mask

https://blog.csdn.net/zhaohongfei_358/article/details/126019181

https://blog.csdn.net/zhaohongfei_358/article/details/122861751

In [ ]:
# 定义编码器，词典大小为10，要把token编码成128维的向量
embedding = nn.Embedding(10, 128)
# 定义transformer，模型维度为128（也就是词向量的维度）
transformer = nn.Transformer(d_model=128, batch_first=True)  # batch_first一定不要忘记
# 定义源句子，可以想想成是 <bos> 我 爱 吃 肉 和 菜 <eos> <pad> <pad>
src = torch.LongTensor([[0, 3, 4, 5, 6, 7, 8, 1, 2, 2]])  # 这个句子有10个词元，每个词元编码为 128维
# 定义目标句子，可以想想是 <bos> I like eat meat and vegetables <eos> <pad>
tgt = torch.LongTensor([[0, 3, 4, 5, 6, 7, 8, 1, 2]])
# 将token编码后送给transformer（这里暂时不加Positional Encoding）
outputs = transformer(embedding(src), embedding(tgt))
outputs.size(), embedding(src).size(), embedding(tgt).size()

In [ ]:
src = torch.LongTensor(
        [
                [0, 8, 3, 5, 5, 9, 6, 1, 2, 2, 2],
                [0, 6, 6, 8, 9, 1, 2, 2, 2, 2, 2],
                ]
        )
tgt = torch.LongTensor(
        [
                [0, 8, 3, 5, 5, 9, 6, 1, 2, 2],
                [0, 6, 6, 8, 9, 1, 2, 2, 2, 2],
                ]
        )


def get_key_padding_mask(tokens):
    key_padding_mask = torch.zeros(tokens.size())
    key_padding_mask[tokens == 2] = -torch.inf
    return key_padding_mask


src_key_padding_mask = get_key_padding_mask(src)
tgt_key_padding_mask = get_key_padding_mask(tgt)
print(tgt_key_padding_mask)

tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size(-1))
print(tgt_mask)

In [ ]:
# 定义编码器，词典大小为10，要把token编码成128维的向量
embedding = nn.Embedding(10, 128)
# 定义transformer，模型维度为128（也就是词向量的维度）
transformer = nn.Transformer(d_model=128, batch_first=True)  # batch_first一定不要忘记
# 将token编码后送给transformer（这里暂时不加Positional Encoding）
outputs = transformer(
        embedding(src), embedding(tgt),
        tgt_mask=tgt_mask,
        src_key_padding_mask=src_key_padding_mask,
        tgt_key_padding_mask=tgt_key_padding_mask
        )
outputs.size(), embedding(src).size(), embedding(tgt).size(),

实战：使用 nn.Transformer 预测输出。
例如：输入为 [0,3,4,6,7,1,2,2]，输出为 [0,3,4,6,7,1]

In [ ]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化Shape为(max_len, d_model)的PE (positional encoding)
        pe = torch.zeros(max_len, d_model)
        # 初始化一个tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里就是sin和cos括号中的内容，通过e和ln进行了变换
        div_term = torch.exp(
                torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
                )
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 为了方便计算，在最外面在unsqueeze出一个batch
        pe = pe.unsqueeze(0)
        # 如果一个参数不参与梯度下降，但又希望保存model的时候将其保存下来
        # 这个时候就可以用register_buffer
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x 为embedding后的inputs，例如(1,7, 128)，batch size为1,7个单词，单词维度为128
        """
        # 将x和positional encoding相加。
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [ ]:
max_length = 16  # 句子的长度
encoding_dim, num_steps = 128, 30
pos_encoding = PositionalEncoding(encoding_dim, 0)
pos_encoding.eval()
X = pos_encoding(torch.zeros((1, num_steps, encoding_dim)))
P = pos_encoding.pe[:, :X.shape[1], :]
X.size(), P.size()

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, d_model=128, dropout=0.5):
        super().__init__()

        self.model_type = 'Transformer'
        self.src_mask = None
        self.embedding = nn.Embedding(num_embeddings=10, embedding_dim=128)
        # 定义位置编码器，词典数为10。我们只预测一位整数。
        self.positional_encoding = PositionalEncoding(d_model, dropout=0)
        # 定义Transformer
        self.transformer = nn.Transformer(
            d_model=128,
            num_encoder_layers=2,
            num_decoder_layers=2,
            dim_feedforward=512,
            batch_first=True
            )

        # 定义最后的线性层，这里并没有用Softmax，因为没必要。
        # 因为后面的CrossEntropyLoss中自带了
        self.predictor = nn.Linear(128, 10)
        pass

    def forward(self, src, tgt):
        # 生成mask
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[-1])
        src_key_padding_mask = TransformerModel.get_key_padding_mask(src)
        tgt_key_padding_mask = TransformerModel.get_key_padding_mask(tgt)

        # 对src和tgt进行编码
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        # 给src和tgt的token增加位置信息
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)

        # 将准备好的数据送给transformer
        out = self.transformer(
            src, tgt,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
            )

        """
        这里直接返回transformer的结果。因为训练和推理时的行为不一样，
        所以在该模型外再进行线性层的预测。
        """
        return out

    @staticmethod
    def get_key_padding_mask(tokens):
        """
        用于key_padding_mask
        """
        key_padding_mask = torch.zeros(tokens.size())
        key_padding_mask[tokens == 2] = -torch.inf
        return key_padding_mask

In [ ]:
src = torch.LongTensor([[0, 3, 4, 5, 6, 1, 2, 2]])
tgt = torch.LongTensor([[3, 4, 5, 6, 1, 2, 2]])
model=TransformerModel()
out = model(src, tgt)
out.size()

In [ ]:
import random

def generate_random_batch(batch_size, max_length=16):
    """生成随机数据

    Args:
        batch_size (_type_): _description_
        max_length (int, optional): _description_. Defaults to 16.

    Returns:
        _type_: _description_
    """
    src = []
    for i in range(batch_size):
        # 随机生成句子长度
        random_len = random.randint(1, max_length - 2)
        # 随机生成句子词汇，并在开头和结尾增加<bos>和<eos>
        random_nums = [0] + [random.randint(3, 9) for _ in range(random_len)] + [1]
        # 如果句子长度不足max_length，进行填充
        random_nums = random_nums + [2] * (max_length - random_len - 2)
        src.append(random_nums)
    src = torch.LongTensor(src)
    # tgt不要最后一个token
    tgt = src[:, :-1]
    # tgt_y不要第一个的token
    tgt_y = src[:, 1:]
    # 计算tgt_y，即要预测的有效token的数量
    n_tokens = (tgt_y != 2).sum()

    # 这里的n_tokens指的是我们要预测的tgt_y中有多少有效的token，后面计算loss要用
    return src, tgt, tgt_y, n_tokens

generate_random_batch(batch_size=2, max_length=6)

In [ ]:
total_loss = 0

for step in range(2000):
    # 生成数据
    src, tgt, tgt_y, n_tokens = generate_random_batch(batch_size=2, max_length=max_length)

    # 清空梯度
    optimizer.zero_grad()
    # 进行transformer的计算
    out = model(src, tgt)
    # 将结果送给最后的线性层进行预测
    out = model.predictor(out)
    """
    计算损失。由于训练时我们的是对所有的输出都进行预测，所以需要对out进行reshape一下。
            我们的out的Shape为(batch_size, 词数, 词典大小)，view之后变为：
            (batch_size*词数, 词典大小)。
            而在这些预测结果中，我们只需要对非<pad>部分进行，所以需要进行正则化。也就是
            除以n_tokens。
    """
    loss = criteria(out.contiguous().view(-1, out.size(-1)), tgt_y.contiguous().view(-1)) / n_tokens
    # 计算梯度
    loss.backward()
    # 更新参数
    optimizer.step()

    total_loss += loss

    # 每40次打印一下loss
    if step != 0 and step % 40 == 0:
        print("Step {}, total_loss: {}".format(step, total_loss))
        total_loss = 0


In [22]:
src = torch.LongTensor([[0, 3, 4, 5, 6, 1, 2, 2]])
tgt = torch.LongTensor([[3, 4, 5, 6, 1, 2, 2]])
model=TransformerModel()
out = model(src, tgt)
out.size()

torch.Size([1, 7, 128])

In [23]:
import random

def generate_random_batch(batch_size, max_length=16):
    """生成随机数据

    Args:
        batch_size (_type_): _description_
        max_length (int, optional): _description_. Defaults to 16.

    Returns:
        _type_: _description_
    """
    src = []
    for i in range(batch_size):
        # 随机生成句子长度
        random_len = random.randint(1, max_length - 2)
        # 随机生成句子词汇，并在开头和结尾增加<bos>和<eos>
        random_nums = [0] + [random.randint(3, 9) for _ in range(random_len)] + [1]
        # 如果句子长度不足max_length，进行填充
        random_nums = random_nums + [2] * (max_length - random_len - 2)
        src.append(random_nums)
    src = torch.LongTensor(src)
    # tgt不要最后一个token
    tgt = src[:, :-1]
    # tgt_y不要第一个的token
    tgt_y = src[:, 1:]
    # 计算tgt_y，即要预测的有效token的数量
    n_tokens = (tgt_y != 2).sum()

    # 这里的n_tokens指的是我们要预测的tgt_y中有多少有效的token，后面计算loss要用
    return src, tgt, tgt_y, n_tokens

generate_random_batch(batch_size=2, max_length=6)

(tensor([[0, 3, 4, 1, 2, 2],
         [0, 3, 9, 1, 2, 2]]),
 tensor([[0, 3, 4, 1, 2],
         [0, 3, 9, 1, 2]]),
 tensor([[3, 4, 1, 2, 2],
         [3, 9, 1, 2, 2]]),
 tensor(6))

In [24]:
total_loss = 0

for step in range(2000):
    # 生成数据
    src, tgt, tgt_y, n_tokens = generate_random_batch(batch_size=2, max_length=max_length)

    # 清空梯度
    optimizer.zero_grad()
    # 进行transformer的计算
    out = model(src, tgt)
    # 将结果送给最后的线性层进行预测
    out = model.predictor(out)
    """
    计算损失。由于训练时我们的是对所有的输出都进行预测，所以需要对out进行reshape一下。
            我们的out的Shape为(batch_size, 词数, 词典大小)，view之后变为：
            (batch_size*词数, 词典大小)。
            而在这些预测结果中，我们只需要对非<pad>部分进行，所以需要进行正则化。也就是
            除以n_tokens。
    """
    loss = criteria(out.contiguous().view(-1, out.size(-1)), tgt_y.contiguous().view(-1)) / n_tokens
    # 计算梯度
    loss.backward()
    # 更新参数
    optimizer.step()

    total_loss += loss

    # 每40次打印一下loss
    if step != 0 and step % 40 == 0:
        print("Step {}, total_loss: {}".format(step, total_loss))
        total_loss = 0


RuntimeError: The size of tensor a (10) must match the size of tensor b (30) at non-singleton dimension 1